In [1]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



/kaggle/input/crowdflower-search-relevance/train.csv.zip
/kaggle/input/crowdflower-search-relevance/sampleSubmission.csv.zip
/kaggle/input/crowdflower-search-relevance/test.csv.zip


In [2]:
data = pd.read_csv("/kaggle/input/crowdflower-search-relevance/train.csv.zip")
data.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471


In [3]:
data.size

60948

In [4]:
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=57495d69b2eefe3528c739611dfddd246d3e7af9425fce74f5faaafd57d5a1dd
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [5]:
from sentence_transformers import SentenceTransformer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
! pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.1 MB/s eta 0:00:00


In [7]:
!pip install pinecone-client -q

In [9]:
import pinecone
pinecone.init(api_key="e9d5474a-6d6c-4336-8925-1984925bdbee", environment="us-west1-gcp-free")
index = pinecone.create_index("crowdflower", dimension=768)
index = pinecone.Index("crowdflower")

In [10]:
from torch.utils.data import DataLoader

model = SentenceTransformer('distilbert-base-nli-mean-tokens')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10158 entries, 0 to 10157
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10158 non-null  int64  
 1   query                10158 non-null  object 
 2   product_title        10158 non-null  object 
 3   product_description  7714 non-null   object 
 4   median_relevance     10158 non-null  int64  
 5   relevance_variance   10158 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 476.3+ KB


In [12]:
data.dropna(inplace=True)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7714 entries, 0 to 10156
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   7714 non-null   int64  
 1   query                7714 non-null   object 
 2   product_title        7714 non-null   object 
 3   product_description  7714 non-null   object 
 4   median_relevance     7714 non-null   int64  
 5   relevance_variance   7714 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 421.9+ KB


In [14]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(data.drop(columns=['median_relevance','relevance_variance'],axis=1),data['median_relevance'])

In [15]:
from sentence_transformers import InputExample

def create_train_set(X,y):
  train_set = []
  for i,row_x in X.iterrows():
    train_set.append(InputExample(texts=[row_x['query'],str(row_x['product_title']+row_x['product_description'])],label=float(y[i])))
  return train_set

In [16]:
X = create_train_set(train_X,train_y)

In [17]:
from sentence_transformers import losses

train_dataloader = DataLoader(X,shuffle=True)
train_loss = losses.CosineSimilarityLoss(model)
model.fit(train_objectives=[(train_dataloader,train_loss)],epochs=1)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5785 [00:00<?, ?it/s]

In [18]:
upserted_data = []
i=0
for _,item in data.iterrows():
  id  = index.describe_index_stats()['total_vector_count']
  upserted_data.append(
      (
        str(id+i),
        model.encode(str(item["product_title"]+item['product_description'])).tolist(),
          {
              'id':item['id'],
              'query':item['query'],
              'title':item['product_title']
          }
      )
  )
  i+=1

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
import math

for i in range(0,math.floor(len(upserted_data)/90)):
    index.upsert(vectors=upserted_data[(i*90):((i+1)*90)])

In [20]:
K = 3

In [21]:
query = "wine rack"
query_em = model.encode(query).tolist()
result = index.query(query_em, top_k=K, includeMetadata=True)
print(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'matches': [{'id': '208',
              'metadata': {'id': 886.0,
                           'query': 'wine rack',
                           'title': 'Anchor Hocking Bamboo Wine Rack'},
              'score': 0.999995232,
              'values': []},
             {'id': '338',
              'metadata': {'id': 1446.0,
                           'query': 'wine rack',
                           'title': '32 Bottle Dakota Wine Rack with Display '
                                    'Top by Wooden Mallet'},
              'score': 0.999995112,
              'values': []},
             {'id': '5504',
              'metadata': {'id': 23401.0,
                           'query': 'wine rack',
                           'title': 'Wildon Home Colombia Glass Wine '
                                    'Rack/Table'},
              'score': 0.999995112,
              'values': []}],
 'namespace': ''}
